In [3]:
# import required packages
import arcpy
from arcpy import env
from arcpy.sa import *
import csv
import pandas as pd
import os

In [4]:
# set workspace - replace this with your project workspace/geodatabase (view HowToFindFilePaths.md on github if confused!)
arcpy.env.workspace = r"C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool"

In [5]:
# view your current working directory
os.getcwd()

'C:\\Users\\andre\\Dropbox\\PC (2)\\Documents\\ArcGIS\\Projects\\OptimalVisibilityTool'

In [6]:
# change your working directory to your ArcGIS project folder
os.chdir(r"C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool")

In [7]:
# view your current working directory to see the new location
cwd = os.getcwd()
print(cwd)

C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool


In [8]:
# set location for raster files - replace this with any folder on your computer that can store your raster files.
# If you aren't sure, put it on your desktop.
rasterFolder = r"C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool\Rasters"
print(rasterFolder)

C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool\Rasters


In [7]:
# set input raster DEM - replace this with your area's DEM! (view HowToFindYourDEM.md on github)
inputRaster = r"C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool\Rasters\SRER.tif"

In [8]:
# boundary polygon for the extent of our area - replace this with a shapefile of your area's boundaries!
# Look up US Parcel boundaries and download a shapefile of your area if you aren't sure
boundaryShapefile = r"C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool\Shapefiles\srerboundary\bounds.shp"

In [9]:
# allows arcpy to overwrite previous outputs
arcpy.env.overwriteOutput = True

In [16]:
# crop raster to boundary to create cropped raster
croppedRaster = ExtractByMask(inputRaster, boundaryShapefile)
# TODO the raster values are missing from the tif file
croppedRaster.save("Rasters/croppedRaster.tif")

In [41]:
# find local peaks within cropped raster.
# Default is 15 points, but set higher for more accuracy, and lower for faster processing time.
optimalPoints = arcpy.defense.FindLocalPeaksValleys(croppedRaster, "optimalVisibilityPoints", "PEAKS", 1)

In [42]:
# Set the path for the new folder to hold visibility outputs relative to the current workspace.
# You can change the folder_name
folder_name = "visibility_output_1"
folder_path = os.path.join(rasterFolder, folder_name)
print(folder_path)

C:\Users\andre\Dropbox\PC (2)\Documents\ArcGIS\Projects\OptimalVisibilityTool\Rasters\visibility_output_1


In [43]:
# Create the new folder if it doesn't already exist
if not os.path.isdir(folder_path):
    try:
        os.mkdir(folder_path)
    except OSError as error:
        print(error)

In [44]:
# select the name of the first field for the optimalVisibilityPoints
field = 'FID'

In [45]:
# list all object IDs from field = 'FID'
all_object_ids = [row[0] for row in arcpy.da.SearchCursor(optimalPoints, field)]
print(all_object_ids)

[0]


In [46]:
# Find the object ID fieldname
objectidfield = arcpy.Describe(feature_class).OIDFieldName
print(objectidfield)

FID


In [47]:
# looping through the points
for pointid in all_object_ids:  # For each object id in the object id list
    
    # create a where clause to select only the point with this object ID
    sql = "{0}={1}".format(
        arcpy.AddFieldDelimiters(
            datasource=optimalPoints, 
            field=objectidfield), 
        pointid)
    
    # create a layer with only this point in it
    arcpy.MakeFeatureLayer_management(
        in_features=optimalPoints, 
        out_layer='tempLayer', 
        where_clause=sql)
    
    # run visibility analysis, with in_raster and templayer as observer
    # observer height is 6 (meters), but you can change the observer_offset value!
    outvis = arcpy.sa.Visibility(croppedRaster, 
                                 'tempLayer', 
                                 analysis_type="OBSERVERS", 
                                 nonvisible_cell_value="NODATA", 
                                 observer_offset=6)
    
    # name the objects using object ID
    # I named them visibility_analysis_1.tif, visibility_analysis_2.tif etc...
    output_raster_name = os.path.join(folder_path,
                                      "visiblity_analysis_{0}.tif".format(pointid))
    # save output raster with name called in above
    outvis.save(output_raster_name)

In [30]:
# Use the ListRasters function to create a list of rasters in the folder
rasters = arcpy.ListRasters()
print(rasters)

['visiblity_analysis_0.tif', 'visiblity_analysis_1.tif', 'visiblity_analysis_2.tif', 'visiblity_analysis_3.tif', 'visiblity_analysis_4.tif', 'visiblity_analysis_5.tif', 'visiblity_analysis_6.tif', 'visiblity_analysis_7.tif', 'visiblity_analysis_8.tif', 'visiblity_analysis_9.tif']


In [31]:
# intialize variables to keep track of the maximum count value and the name of the raster with the maximum count value
max_count = -1
max_count_raster = ""

In [34]:
# Iterate through the list of rasters
for raster in rasters:
    # Create a unique filename for the output dataframe
    output_filename = f"{raster}_table.csv"

    # Get the raster attribute table as a table view
    table_view_name = f"{raster}_view"
    table_view = arcpy.MakeTableView_management(raster, table_view_name)

    # Get the Count value from each raster table using a SearchCursor
    count_field = "Count"
    with arcpy.da.SearchCursor(table_view, count_field) as cursor:
        count_values = [row[0] for row in cursor]

    # Find the maximum count value
    count_max = max(count_values)

    # Check if the Count value is greater than the current maximum count value
    if count_max > max_count:
        # If so, update the maximum count value and the name of the raster with the maximum count value
        max_count = count_max
        max_count_raster = raster

    # Convert the table view to a Pandas dataframe
    # dataframe = pd.DataFrame.from_records(arcpy.da.TableToNumPyArray(table_view, count_field))

    # Save the dataframe to a file
    # dataframe.to_csv(output_filename, index=False)

    # Print a message indicating the dataframe has been saved
    # print(f"{output_filename} saved successfully.")

In [33]:
# Print the name of the raster with the maximum count value
# The count value represents the number of raster cells that the point has visibility of
print(f"Raster with the highest visibility is {max_count_raster} with a count of {max_count}")
# You can find this raster in your specified output folder.
# The raster number corresponds with the point number within the "pointsSelection" feature class
# View the 'HowToInterpretResults.md' in the tutorials folder on github for more information

Raster with the highest visibility is visiblity_analysis_0.tif with a count of 2117230.0
